<a href="https://colab.research.google.com/github/robocarstore/donkey-car-training-on-google-colab/blob/master/Donkey_Car_Training_using_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Check GPU and Tensorflow version
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [ ]:
!nvidia-smi

Verified tensorflow version: > 2.9 and <= 2.15

In [ ]:
import tensorflow
print(tensorflow.__version__)

## Git Clone the donkey repository

In [ ]:
!git clone https://github.com/robocarstore/donkeycar.git
%cd /content/donkeycar

In [ ]:
!git checkout v5

## Install donkey car

In [ ]:
# !pip3 install --upgrade pip setuptools wheel
!pip3 install -e .[pc]

## Create Project

In [ ]:
!donkey createcar --path /content/mycar

### Download your dataset

You can download your dataset to this Colab instance by the following command. You might obtain the uuid of the dataset from the meta.json under each tub.

In [ ]:
%cd /content/mycar/data
!curl  -O -J "https://robocarstore-dc-sg.s3.amazonaws.com/tubs/63783f2f-9497-460b-a6ad-3607c20dcbff/data.tar.gz"


Use the following command to extract the dataset into a folder named tub_xxxxxxx

In [ ]:
!tar -xzf data.tar.gz

Alternatively, use the "Getting UUID" notebook to generate the list of tubs uuid for training

In [ ]:
tub_uuids = ['b7da4c7a-d868-46e1-a2e7-1b02c472c8ef', 'a38a0958-baba-4b65-aded-e60e2f22e265']

In [ ]:
for tub_uuid in tub_uuids:
  !curl -J "https://robocarstore-dc-sg.s3.amazonaws.com/tubs/{tub_uuid}/data.tar.gz" -o /tmp/data.tar.gz
  !tar -zxf /tmp/data.tar.gz -C /content/mycar/data


In [ ]:
!ls

Assuming the latest created folder is the tub_name.

In [ ]:
%cd /content/mycar/data

tub_name=!ls -td -- */ | head -n 1
tub_name = tub_name[0].strip('/')
tub_name

Check whether the data is there

In [ ]:
!ls {tub_name}

Check number of images in this tub

In [ ]:
%cd /content/mycar/data/{tub_name}
!find . -type f -name "*.jpg" | wc -l

### Generate a movie for preview

In [ ]:
!pip uninstall keras-vis
!pip install git+https://github.com/sctse999/keras-vis



In [ ]:
%cd /content/mycar
!donkey makemovie --tub ./data/tub* --out {tub_name}.mp4 --start 1 --end 2000

Download this movie

In [ ]:
from google.colab import files

files.download(f"{tub_name}.mp4")

## Train your model

In [ ]:
%cd /content/mycar

model_name = "model1"

!donkey train --tub ./data/tub* --model models/{model_name}

### Quiz
1. How many records are used for training?
2. How many records are used for validation?
3. Does the sum of this number adds up to the number of images in the dataset?
4. What is the final lost value?
5. How long does this training last?
6. How many epoches in this training?
7. How many steps in this epoch?
8. What's the value of `MAX_EPOCHS` in `myconfig.py`?
9. Why the training stop before it reaches the 100th epoch?

Check if the model is generated



In [ ]:
!ls -alh /content/mycar/models/{model_name}

In [ ]:
%cd /content/mycar/models

import glob
file = glob.glob(f"{model_name}.png")

from IPython.display import Image
Image(file[0])

## Copy the trained model back to Donkey Car (Pi)

Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.

In [ ]:
from google.colab import files

files.download(f"{model_name}.tflite")

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```


## Changing training parameters

### Change number of epoch

1. Edit `myconfig.py`
2. Change `MAX_EPOCHS` to 5, 10 , 20
3. Training the model again by executing the training script above
4. What is `MAX_EPOCHS` used for?
5. Why do we want to change `MAX_EPOCHS`?

### Change TRAIN_FILTER

Change TRAIN_FILTER in `myconfig.py` to > 0.1 and check the number of images used for training and validation

### Change Batch Size

1. Edit `myconfig.py`
2. Change `BATCH_SIZE` to 256
3. Training the model again by executing the training script above
4. Does the training go faster? Why?


## Bonus - Salient Object Visualization

In [ ]:
!pip install imageio-ffmpeg


In [ ]:
%cd /content/mycar
!donkey makemovie --tub data/{tub_name} --model models/{model_name} --type linear --salient  --start 1 --end 1000

t:  59% 3290/5592 [02:04<01:18, 29.38it/s, now=None]

In [ ]:
from google.colab import files

%cd /content/mycar
!ls -ahl
files.download('tub_movie.mp4')